In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Gemma Evaluation

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_gemma_evaluation.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma_evaluation.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates evaluating pre-trained and instruction-tuned Gemma models in Vertex AI.

### Objective

- Evaluate pre-trained and instruction-tuned Gemma model on any of the benchmark datasets
- Clean up the resources

| Models |
| :- |
| [google/gemma-2b](https://huggingface.co/google/gemma-2b)
| [google/gemma-2b-it](https://huggingface.co/google/gemma-2b-it)
| [google/gemma-7b](https://huggingface.co/google/gemma-7b)
| [google/gemma-7b-it](https://huggingface.co/google/gemma-7b-it)
| [google/gemma-1.1-2b-it](https://huggingface.co/google/gemma-1.1-2b-it)
| [google/gemma-1.1-7b-it](https://huggingface.co/google/gemma-1.1-7b-it)

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Run the notebook

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. [Optional] [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

# Import the necessary packages
import os
import json
from datetime import datetime

from google.cloud import aiplatform

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, please change the value yourself below.
now = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_URI = "gs://"  # @param {type:"string"}
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    # Create a unique GCS bucket for this notebook, if not specified by the user
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}"
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )

print(f"Using this GCS Bucket: {BUCKET_URI}")

# Gets the default BUCKET_URI and SERVICE_ACCOUNT if they were not specified by the user.
SERVICE_ACCOUNT = None
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)

# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

# Initialize Vertex AI API.
STAGING_BUCKET = os.path.join(BUCKET_URI, "staging")
MODEL_BUCKET = os.path.join(STAGING_BUCKET, "model")
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

! gcloud config set project $PROJECT_ID

# The evaluation docker image.
EVAL_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-lm-evaluation-harness:20240320_0655_RC00"

# Define common functions


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def get_quota(project_id: str, region: str, resource_id: str) -> int:
  """Returns the quota for a resource in a region. Returns -1 if can not figure out the quota."""
  service_endpoint = "aiplatform.googleapis.com"
  quota_list_output = !gcloud alpha services quota list --service=$service_endpoint  --consumer=projects/$project_id --filter="$service_endpoint/$resource_id" --format=json
  # Use '.s' on the command output because it is an SList type.
  quota_data = json.loads(quota_list_output.s)
  if len(quota_data) == 0 or "consumerQuotaLimits" not in quota_data[0]:
    return -1
  if len(quota_data[0]["consumerQuotaLimits"]) == 0 or "quotaBuckets" not in quota_data[0]["consumerQuotaLimits"][0]:
    return -1
  all_regions_data = quota_data[0]["consumerQuotaLimits"][0]["quotaBuckets"]
  for region_data in all_regions_data:
    if region_data.get('dimensions') and region_data['dimensions']['region'] == region:
      if 'effectiveLimit' in region_data:
        return int(region_data['effectiveLimit'])
      else:
        return 0
  return -1


def get_resource_id(accelerator_type: str, is_for_training: bool) -> str:
  """Returns the resource id for a given accelerator type and the use case.
  Args:
    accelerator_type: The accelerator type.
    is_for_training: Whether the resource is used for training. Set false
    for serving use case.
  Returns:
    The resource id.
  """
  training_accelerator_map = {
      "NVIDIA_TESLA_V100": "custom_model_training_nvidia_v100_gpus",
      "NVIDIA_L4": "custom_model_training_nvidia_l4_gpus",
      "NVIDIA_TESLA_A100": "custom_model_training_nvidia_a100_gpus",
      "NVIDIA_TESLA_T4": "custom_model_training_nvidia_t4_gpus",
      "TPU_V5e": "custom_model_training_tpu_v5e",
      "TPU_V3": "custom_model_training_tpu_v3",
  }
  serving_accelerator_map = {
      "NVIDIA_TESLA_V100": "custom_model_serving_nvidia_v100_gpus",
      "NVIDIA_L4": "custom_model_serving_nvidia_l4_gpus",
      "NVIDIA_TESLA_A100": "custom_model_serving_nvidia_a100_gpus",
      "NVIDIA_TESLA_T4": "custom_model_serving_nvidia_t4_gpus",
      "TPU_V5e": "custom_model_serving_tpu_v5e",
  }
  if is_for_training:
    if accelerator_type in training_accelerator_map:
      return training_accelerator_map[accelerator_type]
    else:
      raise ValueError(
          f"Could not find accelerator type: {accelerator_type} for training."
      )
  else:
    if accelerator_type in serving_accelerator_map:
      return serving_accelerator_map[accelerator_type]
    else:
      raise ValueError(
          f"Could not find accelerator type: {accelerator_type} for serving."
      )


def check_quota(project_id:str, region: str, accelerator_type: str,
                accelerator_count: int, is_for_training: bool):
  """Checks if the project and the region has the required quota."""
  resource_id = get_resource_id(accelerator_type, is_for_training)
  quota = get_quota(project_id, region, resource_id)
  quota_request_instruction = ("Either use "
            "a different region or request additional quota. Follow "
            "instructions here "
            "https://cloud.google.com/docs/quotas/view-manage#requesting_higher_quota"
            " to check quota in a region or request additional quota for "
            "your project.")
  if quota == -1:
    raise ValueError(
            f"""Quota not found for: {resource_id} in {region}.
            {quota_request_instruction}"""
        )
  if quota < accelerator_count:
    raise ValueError(
            f"""Quota not enough for {resource_id} in {region}:
            {quota} < {accelerator_count}.
            {quota_request_instruction}"""
        )

In [ ]:
# @title Evaluate Gemma models

# @markdown This section demonstrates how to evaluate the Gemma models with and without finetuned LoRA adapters using EleutherAI's [Language Model Evaluation Harness (lm-evaluation-harness)](https://github.com/EleutherAI/lm-evaluation-harness) with Vertex CustomJob. Please reference the peak GPU memory usage for serving and adjust the machine type, accelerator type and accelerator count accordingly.

# @markdown You must provide a Hugging Face User Access Token (read) to access the Gemma models. You can follow the [Hugging Face documentation](https://huggingface.co/docs/hub/en/security-tokens) to create a **read** access token and put it in the `HF_TOKEN` field below.
HF_TOKEN = ""  # @param {type:"string", isTemplate:true}

# @markdown This example uses the dataset [HellaSwag](https://arxiv.org/abs/1905.07830). All supported tasks are listed in [this task table](https://github.com/EleutherAI/lm-evaluation-harness/blob/master/docs/task_table.md).
# @markdown Set evaluation dataset.
eval_dataset = "hellaswag"  # @param {type:"string"}

# Worker pool spec.
# Find Vertex AI supported accelerators and regions in:
#  https://cloud.google.com/vertex-ai/docs/training/configure-compute


# Setup evaluation job.
# @markdown Set the base model id.
base_model_id = "google/gemma-1.1-2b-it"  # @param["google/gemma-2b", "google/gemma-2b-it", "google/gemma-7b", "google/gemma-7b-it", "google/gemma-1.1-2b-it", "google/gemma-1.1-7b-it"] {isTemplate:true}
job_name = get_job_name_with_datetime(prefix="gemma-eval")
eval_output_dir = os.path.join(MODEL_BUCKET, job_name)
eval_output_dir_gcsfuse = eval_output_dir.replace("gs://", "/gcs/")

# @markdown Set the accelerator type.
accelerator_type = "NVIDIA_L4"  # @param["NVIDIA_TESLA_V100", "NVIDIA_L4", "NVIDIA_TESLA_A100"]

# @markdown To evaluate a PEFT-finetuned model, enter the PEFT output directory to the LoRA adapter below.
# @markdown Otherwise, leave it empty.
# @markdown See the [finetuning notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma_finetuning_on_vertex.ipynb) for more details.
# @markdown Set the PEFT output directory.
peft_output_dir = ""  # @param {type:"string"}
peft_output_dir_gcsfuse = peft_output_dir.replace("gs://", "/gcs/")

if accelerator_type == "NVIDIA_TESLA_A100":
    machine_type = "a2-highgpu-1g"
    accelerator_count = 1
elif accelerator_type == "NVIDIA_TESLA_V100":
    machine_type = "n1-standard-8"
    accelerator_count = 2
elif accelerator_type == "NVIDIA_L4":
    machine_type = "g2-standard-8"
    accelerator_count = 1
else:
    print(f"Unsupported accelerator type: {accelerator_type}")

replica_count = 1

check_quota(project_id=PROJECT_ID,
            region=REGION,
            accelerator_type=accelerator_type,
            accelerator_count=accelerator_count,
            is_for_training=True)

# Prepare evaluation command that runs the evaluation harness.
# Set `trust_remote_code = True` because evaluating the model requires
# executing code from the model repository.
# Set `use_accelerate = True` to enable evaluation across multiple GPUs.
eval_command = [
    "lm_eval",
    "--model",
    "hf",
    "--tasks",
    f"{eval_dataset}",
    "--output_path",
    f"{eval_output_dir_gcsfuse}",
]

if peft_output_dir_gcsfuse:
    eval_command += [
        "--model_args",
        f"pretrained={base_model_id},peft={peft_output_dir_gcsfuse},trust_remote_code=True,parallelize=True,device_map_option=auto",
    ]
else:
    eval_command += [
        "--model_args",
        f"pretrained={base_model_id},trust_remote_code=True,parallelize=True,device_map_option=auto",
    ]

# Pass evaluation arguments and launch job.
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        },
        "replica_count": replica_count,
        "disk_spec": {
            "boot_disk_size_gb": 500,
        },
        "container_spec": {
            "image_uri": EVAL_DOCKER_URI,
            "env": [
                {
                    "name": "HF_TOKEN",
                    "value": HF_TOKEN,
                }
            ],
            "command": eval_command,
            "args": [],
        },
    }
]

eval_job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    base_output_dir=eval_output_dir,
)

eval_job.run()

print("Evaluation results were saved in:", eval_output_dir)

In [ ]:
# @title Fetch and print evaluation results
import json

from google.cloud import storage

# Fetch evaluation results.
storage_client = storage.Client()
BUCKET_NAME = BUCKET_URI.split("gs://")[1]
bucket = storage_client.get_bucket(BUCKET_NAME)
RESULT_FILE_PATH = eval_output_dir[len(BUCKET_URI) + 1 :] + "/results.json"
blob = bucket.blob(RESULT_FILE_PATH)
raw_result = blob.download_as_string()

# Print evaluation results.
result = json.loads(raw_result)
result_formatted = json.dumps(result, indent=2)
print(f"Evaluation result:\n{result_formatted}")

In [ ]:
# @title Clean up resources
# Delete evaluation job.

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI
    # Uncomment below to delete all artifacts
    # !gsutil -m rm -r $STAGING_BUCKET $MODEL_BUCKET $EXPERIMENT_BUCKET

eval_job.delete()